# ---------------------------- Timelapse Slicer ----------------------------

# todo

utiliser a terme la fonction pour faire le timelapse avec les bandes de décalage
faire partir de la photo de base, mais démarrer progressivement bande par bande, finir progressivement bande par bande
faire aussi décaler les bandes progressivement


fonction avec proportion du centre (cliquable?)

heure qui défile sur le timelapse

# Principe
On va réaliser un collage à partir d'un dossier contenant un ensemble de photos d'un timelapse réalisé sans mouvement de l'appareil :

La photo est constituée de bandes verticales, chacune provenant d'une photo différente, mais respectant son emplacement (on ne revoit pas la même scène x fois)

Selon l'endroit sur la photo finale, on verra un moment différent du timelapse.

Différents modes existent :
        
        0 : centré sur le début  (le milieu de la photo est constitué de bandes issues du début du timelapse)
        
        1 : centré sur la fin
        
        2 : de gauche à droite
        
        3 : de droite à gauche

Les variables globales ci-dessous doivent être assignées avant le traitement:

        inputStr, outputStr : les adresse du dossier des photos à utiliser et de celui de l'export

        num_premiere_photo, derniere_photo : les limites des photos à utiliser dans le dossier inputStr
        (la fonction compteur_de_photos(inputStr) renvoie le nombre de photos contenues)

        bandes : le nombre de bandes souhaitées sur la composition

        affichage progressif = 1 permet de suivre la composition collage par collage, mettre à 0 sinon
        
        affichage final = 1 permet d'afficher la composition une fois terminée

        ouvrir_dossier_sortie = 1 ouvre le dossier contenant la composition


On peut réaliser une composition avec la fonction composition(...) mais une fonction composition_itere existe :
elle permet de créer une série de compositions en faisant varier le nombre de bandes de 1 à 'bandes', le nombre de bandes varie en fonction des variable suivantes :

        facteur_itération : facteur de multiplication du nombre de bandes entre deux compositions, peut être un flottant
        pas_iteration : nombre de bandes de différence entre deux compositions, est pris en compte si le facteur=0        



In [3]:
inputStr=r"\input" #pas besoin de gérer les / et \
outputStr=r"\output"

num_premiere_photo = 0
num_derniere_photo = 5      #=compteur_de_photos(inputStr)
bandes=990                      # passé à l'impair supérieur pour les version centrées

affichage_progressif=0
affichage_final=1
ouvrir_dossier_sortie=1
facteur_iteration = 1.4          #mettre au moins une des deux valeurs !=0, facteur prioritaire au pas
pas_itération = 0

### exécution finale : (commenter pour le run all)

###### imports et adaptation des adresses

In [4]:
from pathlib import Path
import PIL
import os
from PIL import Image, ImageDraw,ImageFont
from PIL.PngImagePlugin import PngInfo


inputStr=inputStr.replace('\\', "/")
outputStr=outputStr.replace('\\', "/")

In [5]:
def compteur_de_photos(inputStr):
    """Fonction qui compte les éleements d'un dossier
    input : 
        inputStr : string de l'adresse du dossier dont on veut dénombrer le contenu
    output :
        nombre entier d'éléments contenus
    """
    num_derniere_photo = 0
    inputStr=inputStr.replace('\\', "/")
    for file in os.listdir(inputStr):
        num_derniere_photo += 1
    return num_derniere_photo

In [6]:
def folder(addr,rm=1):
    """Procédure qui crée le dossier si besoin, le vide si on veut
    inputs :
        addr : string de l'adresse du dossier à créer/vérifier
        rm = 1-> on vide le contenu, rm = 0 -> on ne le vide pas
    utilise la procédure creerDossiers
    """
    if os.path.exists(addr):
        if rm==1:
            for pic in os.listdir(addr):
                os.remove(addr+"/"+pic)
    else:
        creerDossiers(addr)
        
def creerDossiers(addr):
    """procédure récursive de création de l'arborescence des dossiers
    inputs :
        addr : string de l'adresse du dossier à créer
    """
    if not os.path.exists(addr):
        i=len(addr)
        while addr[i-1]!="/" and i>0:  #conditons limites
            i-=1
        if i==0:
            print("la racine n'existe pas")
        else:
            parent = addr[:i-1]  #on teste le dossier parent dans l'adresse
            creerDossiers(parent)
            if os.path.exists(parent):
                os.mkdir(addr)



In [7]:
def ajustement_nb_photos(mode,nb_bandes,num_derniere_photo,num_premiere_photo): 
    """fonction qui ajuste si besoin le nombre de bandes,
    calcule le nombre de photos à découper,
    calcule l'indice de la dernière photo à utiliser pour que le compte tombe exactement à la première photo lors de la dernière itération
    
    inputs : 
        mode : mode d'assemblage (0 : centré_sur_le_début
                                  1 : centré_sur_la_fin
                                  2 : de_gauche_à_droite
                                  3 : de_droite_à_gauche)
        nb_bandes : nombre de bandes que l'on souhaite sur la photo
        num_derniere_photo : indice de la dernière photo que l'on veut utiliser pour le découpage/assemblage
        num_premiere_photo : indice de la premiere photo que l'on veut utiliser pour le découpage/assemblage
    outputs :
        nb_photos : nombre de photos que l'on va utiliser pour le découpage/assemblage
        num_derniere_photo_opti : indice de la dernière photo à utiliser pour que le compte tombe exactement à la première photo lors de la dernière itération
        nb_bandes : nombre de bandes qui seront réellement utilisés sur le traitement
    """
    if mode<=1: #centré
        if nb_bandes%2==0:
            nb_bandes+=1  #permet une symétrie
        nb_photos=int((nb_bandes-1)/2+1)
    else:  #linéaire
        nb_photos=nb_bandes  
    num_derniere_photo_opti = num_derniere_photo-(num_derniere_photo-num_premiere_photo)%(nb_photos-1)            #numéro dernière photo #here

    return nb_photos,num_derniere_photo_opti,nb_bandes

In [8]:
def horloge(img):
    """fonction qui retourne l'heure de prise de l'image"""
    date=img._getexif()[36867]
    heures = date[-9:-6]
    minutes=date[-5:-3]
    Heure=str(heures)+"h"+str(minutes)
    return Heure

In [9]:
#fonction de somme de tuples
tuple_add = lambda a, b: tuple(i+j for i, j in zip(a, b))

In [10]:
def paste_heure(overlay,fond,largeur_bande,corner):
    heure=horloge(overlay)
    I1 = ImageDraw.Draw(fond)
    taille_police=largeur_bande//8
    myFont = ImageFont.truetype('Calibri_Regular.ttf', taille_police)
    I1.text(tuple_add(corner,(largeur_bande//2-(taille_police//2)*3,100)), heure, font=myFont, fill =(255, 255, 255))
    return fond

In [11]:
def choix_fond(mode,num_derniere_photo_opti,num_premiere_photo):  #inputstr
    """fonction qui choisit l'arrière plan : 
            la première ou dernière image pour les version centrées
            moitié moitié pour les versions linéaires en suivant le sens
    Par la suite, on viendra coller sur ce fond les différentes bandes sauf les extérieures, ainsi les résidus de division sont répartis sur les bords
    Les bandes extérieures sont donc plus larges. On peut rogner avec la fonction rognage (pas souhaitable si on veut faire défiler les photos)
    inputs:
        mode : mode d'assemblage (0 : centré_sur_le_début
                                  1 : centré_sur_la_fin
                                  2 : de_gauche_à_droite
                                  3 : de_droite_à_gauche)
        num_derniere_photo_opti : indice de la dernière photo que l'on veut utiliser pour le découpage/assemblage
        num_premiere_photo : indice de la premiere photo que l'on veut utiliser pour le découpage/assemblage
    outputs:
        fond : image de fond
        largeur, hauteur : dimensions du fond
    """
    if mode== 0:
        num_fond=num_derniere_photo_opti
    else:
        num_fond=num_premiere_photo
    fond = Image.open(inputStr+"/"+os.listdir(inputStr)[num_fond])
    largeur, hauteur = fond.size
    
    if mode ==2: # gd
        overlay = Image.open(inputStr+"/"+os.listdir(inputStr)[num_derniere_photo_opti])  
        overlaycrop = overlay.crop((largeur//2,0,largeur,hauteur))
        corner=(largeur//2,0)
        fond.paste(overlaycrop,corner)
    elif mode ==3: # dg
        overlay = Image.open(inputStr+"/"+os.listdir(inputStr)[num_derniere_photo_opti])  
        overlaycrop = overlay.crop((0,0,largeur//2,hauteur))
        corner=(0,0)
        fond.paste(overlaycrop,corner)
    
    return fond, largeur, hauteur



In [12]:
def preparation_decoupage(largeur, nb_bandes, num_derniere_photo_opti,nb_photos,num_premiere_photo):
    """fonction prépare le découpage
    inputs :
        largeur : largeur de la photo
        nb_bandes : nombre de bandes à couper
        num_derniere_photo_opti : indice de la dernière photo que l'on veut utiliser pour le découpage/assemblage
        nb_photos : nombre de photos à utiliser
        num_premiere_photo : indice de la premiere photo que l'on veut utiliser pour le découpage/assemblage
    outputs : 
        largeur_bande : largeur des bandes à couper
        pas_photos : nombre de fichiers entre deux photos importées
        espace_residuel : résidu des divisions, seront répartis sur les bandes extérieures (éventuellement coupées par la fonction rognage)
    """
    largeur_bande = largeur//nb_bandes                                             
    pas_photos=(num_derniere_photo_opti-num_premiere_photo)//(nb_photos-1)
    espace_residuel=int((largeur%(nb_bandes))/2)                                
    return largeur_bande,pas_photos,espace_residuel

In [13]:
def choix_photo_iteration(mode,nb_photos,pas_photos,n_iteration,num_premiere_photo):
    """fonction choisit la photo à utiliser
    inputs:
        mode : mode d'assemblage (0 : centré_sur_le_début
                                  1 : centré_sur_la_fin
                                  2 : de_gauche_à_droite
                                  3 : de_droite_à_gauche)
        nb_photos : nombre de photos à utiliser dans l'assemblage
        pas_photos : nombre de fichiers entre deux photos importées
        n_iteration : itération en cours
        num_premiere_photo : indice de la premiere photo que l'on veut utiliser pour le découpage/assemblage

    output:
        indice de la photo à importer
    """
    if mode ==0 or mode== 3:
        return num_premiere_photo+pas_photos*(nb_photos-n_iteration-1)  #commence par la fin
    else :
        return num_premiere_photo+pas_photos*(n_iteration)  #commence par le debut

In [14]:
def overlay_cropNpaste(fond,overlay,mode,espace_residuel,largeur,largeur_bande,n_iteration,hauteur,print_heure=0):
    """fonction qui réalise le découpage et collage des bandes, appelée aa chaque itération
    inputs : 
        fond : image de fond
        overlay : image à découper/coller
        mode : mode d'assemblage (0 : centré_sur_le_début
                                  1 : centré_sur_la_fin
                                  2 : de_gauche_à_droite
                                  3 : de_droite_à_gauche)
        espace_residuel : résidu des divisions, seront répartis sur les bandes extérieures (éventuellement coupées par la fonction rognage)
        largeur : largeur du fond
        largeur_bande : largeur de la bande à coller
        n_iteration itération en cours
        hauteur : hauteur du fond

    outputs : 
        fond : image avec le collage de l'itération en cours réalisé
    """
    if mode<=1:
        overlaycrop1 = overlay.crop((espace_residuel+largeur_bande*n_iteration,0,espace_residuel+largeur_bande*(n_iteration+1),hauteur))
        overlaycrop2 = overlay.crop((largeur-espace_residuel-(n_iteration+1)*largeur_bande,0,largeur-espace_residuel-n_iteration*largeur_bande,hauteur))
        corner1=(espace_residuel+largeur_bande*n_iteration,0)
        corner2=(largeur-espace_residuel-(n_iteration+1)*largeur_bande,0)
        fond.paste(overlaycrop1,corner1)
        fond.paste(overlaycrop2,corner2)
        if print_heure:
            paste_heure(overlay,fond,largeur_bande,corner1)
            paste_heure(overlay,fond,largeur_bande,corner2)
    else:
        overlaycrop = overlay.crop((espace_residuel+largeur_bande*n_iteration,0,espace_residuel+largeur_bande*(n_iteration+1),hauteur))
        corner=(espace_residuel+largeur_bande*n_iteration,0)
        fond.paste(overlaycrop,corner)
        if print_heure:
            paste_heure(overlay,fond,largeur_bande,corner)
    return fond

In [15]:
def rognage_residus(image,residu,largeur,hauteur):
    """fonction qui coupe les résidus du proces qui sont sur les extérieurs
        permet une symétrie, à éviter si on veut faire défiler les photos
    input : 
        image : fond à découper
        residu : marge à couper en tout (la moitié de chaque coté)
        largeur, hauteur : dimensions de l'Image
    output :
        image : image découpée
    """
    image=image.crop((residu//2,0,largeur-residu//2,hauteur))
    return image

In [16]:
def pngInfoWriter():
    """creation du texte avec les propriétés utilisés, inclus dans le fichier png
    
    Pour le lire à partir d'un fichier : 
        targetImage = Image.open(outputImg)
        print(targetImage.text)
    """
    texte = "num_premiere_photo = "+str(num_premiere_photo)
    texte+= ", num_derniere_photo = "+str(num_derniere_photo)
    texte+= ", bandes = "+str(bandes)
    metadata = PngInfo()
    metadata.add_text("Propriétés",texte)
    return metadata

In [17]:
def composition(nb_bandes,num_derniere_photo,num_premiere_photo,itere=0,mode=0,premiere_iteration=0,rognage=0):
    """fonction principale pour réaliser une composition entière sur une photo, le fichier est sauvegardé en .png
    utilise les variables globales définies et assignées en première case: 
            inputStr
            outputStr
            affichage_progressif
            affichage_final
            ouvrir_dossier_sortie
    inputs : 
        nb_bandes : nombre de bandes à couper
        num_derniere_photo : indice de la dernière photo que l'on veut utiliser pour le découpage/assemblage
        num_premiere_photo : indice de la premiere photo que l'on veut utiliser pour le découpage/assemblage
        itere : 0 ce traitement n'appartient pas à une série de traitements
                1 ce traitement appartient à une série de traitements
        mode : mode d'assemblage (0 : centré_sur_le_début
                                  1 : centré_sur_la_fin
                                  2 : de_gauche_à_droite
                                  3 : de_droite_à_gauche)
        premiere_iteration : 0 si ce traitement appartient à une série de traitements, ce n'est pas la première itération
                             1 si ce traitement appartient à une série de traitements, c'est la première itération
        rognage : 0 on ne rognera pas les résidus
                  1 on rognera les résidus
        
        output : 
            renvoie l'adresse de l'image sauvegardée
    """
    if nb_bandes==1:            #on se passe des différentes opérations s'il n'y a qu'une bande (évite des divisions par 0)
        if mode==1:
            numfond = num_premiere_photo         
        else:
            numfond=num_derniere_photo
        fond = Image.open(inputStr+"/"+os.listdir(inputStr)[numfond])
        print("nb photo", 1)
        print("num photo", numfond)
        if affichage_progressif:
                fond.show()
    else:  #plusieurs bandes
        nb_photos,num_derniere_photo_opti,nb_bandes = ajustement_nb_photos(mode,nb_bandes,num_derniere_photo,num_premiere_photo)
        fond,largeur,hauteur = choix_fond(mode,num_derniere_photo_opti,num_premiere_photo)                                 
        
        print("num opti",num_derniere_photo_opti)
        print("nb photo", nb_photos)
        print("num premiere photo ", num_premiere_photo)

        largeur_bande,pas_photos,espace_residuel=preparation_decoupage(largeur, nb_bandes, num_derniere_photo_opti,nb_photos,num_premiere_photo)
        print("pas photo :", pas_photos)
        for i in range(1,nb_photos):  #on itère sur toutes les photos à couper/coller
            photo_utilisee=choix_photo_iteration(mode,nb_photos,pas_photos,i,num_premiere_photo)
            print("photo utilisée",photo_utilisee)
            overlay = Image.open(inputStr+"/"+os.listdir(inputStr)[photo_utilisee])
            fond=overlay_cropNpaste(fond,overlay,mode,espace_residuel,largeur,largeur_bande,i,hauteur)
            if affichage_progressif:
                fond.show()
            print(str(round(100*(i)/nb_photos))+" %")  #avancement du traitement de la photo
        print("100 %")
    if rognage:
        fond = rognage_residus(fond,espace_residuel,largeur,hauteur)
    noms_modes=["centré_sur_le_début","centré_sur_la_fin","de_gauche_à_droite","de_droite_à_gauche"]
    if itere:
        outputImgAddr = outputStr+"/itéré/"+noms_modes[mode]
        if premiere_iteration:
            folder(outputImgAddr,rm=1) #préparation du dossier
    else:
        print("terminé !")
        outputImgAddr = outputStr+"/seul/"+noms_modes[mode]
        folder(outputImgAddr,rm=0)
    outputImg = outputImgAddr+"/"+str(nb_bandes)+"_bandes.png"
    
    metadata=pngInfoWriter()
    fond.save(outputImg, pnginfo=metadata)   #sauvegarde de la composition
    if affichage_final and not affichage_progressif:
        os.startfile(outputImg)
    if not itere and ouvrir_dossier_sortie:
        os.startfile(outputImgAddr)
    return(outputImgAddr)


In [18]:
def composition_itere(mode_utilise,num_derniere_photo,num_premiere_photo):
    """procédure d'itération du découpage en faisant varier le nombre de bandes, démarre à 1
    utilise les variables globales définies et assignées en première case :
                        bandes
                        num_derniere_photo
                        facteur_iteration
                        pas_itération
    inputs : 
        mode : mode d'assemblage (0 : centré_sur_le_début
                                  1 : centré_sur_la_fin
                                  2 : de_gauche_à_droite
                                  3 : de_droite_à_gauche)
        num_premiere_photo : indice de la premiere photo que l'on veut utiliser pour le découpage/assemblage                    

    """
    bandes_iteration = 1
    while bandes_iteration<=bandes:
        outputImgAddr=composition(round(bandes_iteration),num_derniere_photo,num_premiere_photo,itere=1,mode=mode_utilise,premiere_iteration=bandes_iteration==1)
        if facteur_iteration>0:
            bandes_iteration=bandes_iteration*facteur_iteration
        elif pas_itération>0:
            bandes_iteration+=pas_itération
        else:
            bandes_iteration=bandes+1 #pour sortir du while
            print("ERREUR changer le pas ou le facteur d'itération")
    print("terminé!")
    if ouvrir_dossier_sortie:
        os.startfile(outputImgAddr)